In [1]:
import pandas as pd
import numpy as np

data_path = "../data/binding-blade/"
unwanted_columns = ["Unnamed: 0.1", "Unnamed: 0"]
bases = pd.read_csv(f"{data_path}base-stats.csv").drop(unwanted_columns, axis=1, errors="ignore")
growths = pd.read_csv(f"{data_path}growth-rates.csv").drop(unwanted_columns, axis=1, errors="ignore")

display(bases.head())
display(growths.head())

,Name,Class,Lv,HP,S/M,Skl,Spd,Lck,Def,Res,Con,Mov,Affin,Weapon ranks,bonus_character
0,Allen,Cavalier,1,21,7,4,6,3,6,0,9,7,Fire,"['E Sword', 'D Lance']",False
1,Astohl,Thief,10,25,7,8,15,11,7,3,8,6,Dark,['C Sword'],False
2,Barth,Knight,9,25,10,6,5,2,14,1,16,4,Ice,['C Lance'],False
3,Bartre,Warrior,2,48,22,11,10,14,10,3,14,6,Thunder,"['A Axe', 'D Bow']",False
4,Bors,Knight,1,20,7,4,3,4,11,0,14,4,Wind,['C Lance'],False


,Name,HP,S/M,Skl,Spd,Lck,Def,Res
0,Roy,80,40,50,40,60,25,30
1,Marcus,60,25,20,25,20,15,20
2,Allen,85,45,40,45,40,25,10
3,Lance,80,40,45,50,35,20,15
4,Wolt,80,40,50,40,40,20,10


# Step 1: Create Weapon Rank DataFrame

In [2]:
from ast import literal_eval

def handle_ranks(ranks):
    """
    Functions goal is to take the weapon rank column like "["A Sword", "B Axe"]"
    and convert it into a format I can actually work with. 
    
    Function also standardizes the missing weapon ranks.
    """
    if (isinstance(ranks, str)) and (ranks != "–"):
        ranks = literal_eval(ranks)
    elif (isinstance(ranks, float) and np.isnan(ranks)) or ranks == "–":
        ranks = []  
    return ranks

bases["weapons"] = bases["Weapon ranks"].apply(handle_ranks)
weapon_df = bases[['Name', "weapons"]]
weapon_df = weapon_df.explode("weapons").dropna()
weapon_df[['Rank', 'Weapon']] = weapon_df['weapons'].str.split(' ', expand=True)
weapon_df.drop(columns = "weapons", inplace=True)
weapon_df.to_csv(f"{data_path}weapon-ranks.csv", index=False)

weapon_df.head()

,Name,Rank,Weapon
0,Allen,E,Sword
0,Allen,D,Lance
1,Astohl,C,Sword
2,Barth,C,Lance
3,Bartre,A,Axe


# Step 2: Handle Bonus Characters

In [3]:
bonus_characters = ["Narshen", "Gale", "Hector", "Brunya", "Eliwood", "Murdoch", "Zephiel", "Guinevere"]
bases["bonus_character"] = bases.apply(lambda x: x["Name"] in bonus_characters, axis=1)
bases.drop("weapons", axis=1, inplace=True)
bases.to_csv(f"{data_path}base-stats.csv", index=False)

bases.head()

,Name,Class,Lv,HP,S/M,Skl,Spd,Lck,Def,Res,Con,Mov,Affin,Weapon ranks,bonus_character
0,Allen,Cavalier,1,21,7,4,6,3,6,0,9,7,Fire,"['E Sword', 'D Lance']",False
1,Astohl,Thief,10,25,7,8,15,11,7,3,8,6,Dark,['C Sword'],False
2,Barth,Knight,9,25,10,6,5,2,14,1,16,4,Ice,['C Lance'],False
3,Bartre,Warrior,2,48,22,11,10,14,10,3,14,6,Thunder,"['A Axe', 'D Bow']",False
4,Bors,Knight,1,20,7,4,3,4,11,0,14,4,Wind,['C Lance'],False


# Step 3: Fix Affinity Column

In [4]:
rerun = False
if rerun:
    pattern = r"\[\'(.*?)\'\]"
    bases["Affin"] = bases["Affin"].str.extract(pattern)
    bases.to_csv(f"{data_path}base-stats.csv")
    bases.head()